In [68]:
import numpy as np
import skimage.io as sio
import cv2

from matplotlib import pyplot as plt
%matplotlib inline

from PIL import Image
import copy
from pyquaternion import Quaternion

from nuscenes.nuscenes import NuScenes
from nuscenes.utils.data_classes import LidarPointCloud

import os
import os.path as osp
import json
import tqdm

In [69]:
nusc = NuScenes(version='v1.0-mini', dataroot='../shubham/', verbose=True) 

Loading NuScenes tables for version v1.0-mini...
23 category,
8 attribute,
4 visibility,
911 instance,
12 sensor,
120 calibrated_sensor,
31206 ego_pose,
8 log,
10 scene,
404 sample,
31206 sample_data,
18538 sample_annotation,
4 map,
Done loading in 0.734 seconds.
Reverse indexing ...
Done reverse indexing in 0.1 seconds.


In [70]:
def translation2transform(vec):
    i = np.eye(4)
    i[:3, -1] = vec
    return i

In [71]:
scene_path = "../sus_data/even_smaller_scene/"
img_path = os.path.join(scene_path, 'images')
pcd_path = os.path.join(scene_path, 'pcd')

In [72]:
from datasets import dataset_dict

In [73]:
print(scene_path)

../sus_data/even_smaller_scene/


In [74]:
dataset = dataset_dict['nusc']("../sus_data/even_smaller_scene/", img_wh=(400, 225))
dataset_val = dataset_dict['nusc']("../sus_data/even_smaller_scene/", img_wh=(400, 225), split="val")


In [75]:
n_vals = 90000
which_cam = 1
rays_o, rays_d = dataset.all_rays[which_cam * n_vals:n_vals*(which_cam+1), :3], dataset.all_rays[which_cam*n_vals:n_vals*(which_cam+1), 3:6]
near, far = dataset.all_rays[which_cam * n_vals:n_vals*(which_cam+1), 6:7], dataset.all_rays[which_cam*n_vals:n_vals*(which_cam+1), 7:8]

start = (rays_o + near * rays_d).numpy()
end = (rays_o + far * rays_d).numpy()

In [76]:
print(start[0], end[0])

[ -1.677002 -35.963867   1.02    ] [-6.4750664e+04 -3.2683334e+05  1.9900000e+02]


In [77]:
import plotly.graph_objects as go


In [78]:
# First define list of images
all_cams = ['CAM_FRONT', 'CAM_FRONT_RIGHT', 'CAM_FRONT_LEFT']#, 'CAM_BACK_RIGHT', 'CAM_BACK', 'CAM_BACK_LEFT']

# intrinsic = {}
# extrinsic = {}

# for ix in all_cams:
#     intrinsic[ix] = []
#     extrinsic[ix] = []

full_dict = []

downscale = 4


for n_idx in tqdm.tqdm_notebook(range(22)):
    # Get a particular sample
    my_sample = nusc.sample[n_idx]
    my_sample_token = my_sample['token']
    sample_record = nusc.get('sample', my_sample_token)

    # Get point sensor token
    pointsensor_token = sample_record['data']['LIDAR_TOP']

    # Get point cloud
    pointsensor = nusc.get('sample_data', pointsensor_token)
    pcl_path = os.path.join(nusc.dataroot, pointsensor['filename'])
    if n_idx == 0:
        pc = LidarPointCloud.from_file(pcl_path)

    # Iterate through the list
    for cam_ix in all_cams:
        img_folder = os.path.join(img_path, cam_ix)
        # os.makedirs(img_folder, exist_ok=True)
        img_save_path = os.path.join("{}_{}".format(img_folder, str(n_idx).zfill(3)) + '.jpg')

        # Get the camera token
        camera_token = sample_record['data'][cam_ix]
        cam = nusc.get('sample_data', camera_token)
        im = Image.open(osp.join(nusc.dataroot, cam['filename']))
        w, h = im.size
        
        res_w, res_h = w//downscale, h//downscale
        res_mat = np.eye(3)
        res_mat[0, 0] = res_mat[1, 1] = 1/downscale
        
        # Save the image to the location
        # im = im.resize((res_w, res_h), Image.LANCZOS)
        # im.save(img_save_path)
        # print(img_save_path)
        
        
        # Commpute the calibration matrices (Don't save right now)
        cs_record = nusc.get('calibrated_sensor', pointsensor['calibrated_sensor_token'])
        R1 = Quaternion(cs_record['rotation']).transformation_matrix
        T1 = translation2transform(np.array(cs_record['translation']))

        if n_idx == 0 and cam_ix == 'CAM_FRONT_RIGHT':
            pc.rotate(Quaternion(cs_record['rotation']).rotation_matrix)
            pc.translate(np.array(cs_record['translation']))

        # Second step: transform from ego to the global frame.
        poserecord = nusc.get('ego_pose', pointsensor['ego_pose_token'])
        R2 = Quaternion(poserecord['rotation']).transformation_matrix
        T2 = translation2transform(np.array(poserecord['translation']))

        if n_idx == 0 and cam_ix == 'CAM_FRONT_RIGHT':
            pc.rotate(Quaternion(poserecord['rotation']).rotation_matrix)
            pc.translate(np.array(poserecord['translation']))

        # Third step: transform from global into the ego vehicle frame for the timestamp of the image.
        poserecord = nusc.get('ego_pose', cam['ego_pose_token'])
        T3 = translation2transform(-np.array(poserecord['translation']))
        R3 = Quaternion(poserecord['rotation']).transformation_matrix.T

        # Fourth step: transform from ego into the camera.
        cs_record = nusc.get('calibrated_sensor', cam['calibrated_sensor_token'])
        T4 = translation2transform(-np.array(cs_record['translation']))
        R4 = Quaternion(cs_record['rotation']).transformation_matrix.T
        
        # intrinsic[cam_ix].append(np.array(cs_record['camera_intrinsic']).flatten())
        # extrinsic[cam_ix].append((np.linalg.inv(R4 @ T4 @ R3 @ T3)).flatten())
        intrinsic = (res_mat @ np.array(cs_record['camera_intrinsic'])).flatten()
        extrinsic = (np.linalg.inv(R4 @ T4 @ R3 @ T3)).flatten()
        
        # np.set_printoptions(suppress=True)
        
        data_dict = {
            'intrinsic': list(intrinsic),
            'extrinsic': list(extrinsic),
            'file_name': "{}/{}_{}".format("images", cam_ix, str(n_idx).zfill(3))
        }
        
        full_dict.append(data_dict)

<ipython-input-78-f082edaa7382>:16: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for n_idx in tqdm.tqdm_notebook(range(22)):


  0%|          | 0/22 [00:00<?, ?it/s]

In [79]:
end[::500].shape

(180, 3)

In [80]:
def read_meta(root_dir):
    with open(os.path.join(root_dir,
                            f"transforms_train.json"), 'r') as f:
        meta = json.load(f)
    
    near = 2.0
    far = 10.0
    bounds = np.array([near, far])
    w, h = 800, 450
    
    all_Ks = []
    all_poses = []
    for t, frame in enumerate(meta['frames']):
        # if t >= 40:
        #     break
        all_Ks.append(np.array(frame['intrinsic']).reshape((3, 3)))
        all_poses.append(np.linalg.inv(np.array(frame['extrinsic']).reshape((4, 4)))[:3, :4])
    
    return all_Ks, all_poses
        

In [81]:
# Ks, poses = read_meta("../sus_data/scaled_scene/")

# poses = np.array(poses)
# poses.shape

In [82]:
poses = []
Ks = []
for ix in full_dict:
    poses.append(np.array(ix['extrinsic']).reshape((4, 4)))
    Ks.append(np.array(ix['intrinsic']).reshape((3, 3)))
poses = np.array(poses)
Ks = np.array(Ks)

In [83]:
pc.points[0, ::10].shape

(3469,)

In [84]:
Ks[0]

array([[316.60430076,   0.        , 204.06675494],
       [  0.        , 316.60430076, 122.87676645],
       [  0.        ,   0.        ,   1.        ]])

In [85]:
poses[0][:3, :4][:, :3].T

array([[-0.94016521,  0.33999683,  0.02217038],
       [-0.01558255,  0.02209463, -0.99963444],
       [-0.34036239, -0.940167  , -0.01547459]])

In [86]:
import torch

In [87]:
Ks[0]

array([[316.60430076,   0.        , 204.06675494],
       [  0.        , 316.60430076, 122.87676645],
       [  0.        ,   0.        ,   1.        ]])

In [88]:
from kornia import create_meshgrid

In [125]:
grid = create_meshgrid(100, 100, normalized_coordinates=False)[0]
i, j = grid.unbind(-1)

print(i.shape)

torch.Size([100, 100])


In [126]:
foc = 312

In [127]:
dirs = -1*torch.stack([(i-100/2)/foc, -(j-100/2)/foc, -torch.ones_like(i)], -1)
print(dirs.reshape((-1, 3)).mean(0))

tensor([ 0.0016, -0.0016,  1.0000])


In [129]:
poses[0][:3, :3].T

array([[-0.94016521,  0.33999683,  0.02217038],
       [-0.01558255,  0.02209463, -0.99963444],
       [-0.34036239, -0.940167  , -0.01547459]])

In [130]:
c2w = torch.FloatTensor(poses[0][:3, :4])
rays_d = dirs @ c2w[:, :3].T
print(rays_d.reshape((-1, 3)).mean(0))

rays_d = rays_d / torch.norm(rays_d, dim=-1, keepdim=True)
print(rays_d.reshape((-1, 3)).mean(0))

rays_o = c2w[:, 3].expand(rays_d.shape)
pts = rays_d.reshape((-1, 3)).data.numpy()
pts2 = rays_o.reshape((-1, 3)).data.numpy()
print(pts.shape, pts2.shape)

tensor([-0.3418, -0.9397, -0.0138])
tensor([-0.3390, -0.9318, -0.0137])
(10000, 3) (10000, 3)


In [112]:
p1 = pts2 + pts
p2 = pts2 + 10*pts

In [110]:
pts.mean(0), pts2.mean(0)

(array([0.33895683, 0.9317582 , 0.01373428], dtype=float32),
 array([ 410.81375  , 1179.6614   ,    1.4938738], dtype=float32))

In [105]:
import ipyvolume as ipv

In [117]:
ipv.figure()

# ipv.scatter(pts[:, 0], pts[:, 1], pts[:, 2])#, size=0.1)
# ipv.scatter(pts2[:, 0], pts2[:, 1], pts2[:, 2])#, size=0.1)

ipv.scatter(p1[::5, 0]/100, p1[::5, 1]/100, p1[::5, 2])
ipv.scatter(p2[::5, 0]/100, p2[::5, 1]/100, p2[::5, 2])

# ipv.xlim(0, 1000)
# ipv.ylim(0, 1000)
# ipv.zlim(0, 1000)

ipv.show()

In [96]:
# am, aM = pc.points.min(1), pc.points.max(1)

# print(am.shape, aM.shape)

In [119]:
fig = go.Figure()

skip = 3

fig.add_trace(
    go.Scatter3d(
        x=pc.points[0, ::skip]/2000,
        y=pc.points[1, ::skip]/2000,
        z=pc.points[2, ::skip]/2000,
        mode='markers',
        name='scene',
        marker=dict(size=0.9)
    )
)

fig.add_trace(
    go.Scatter3d(
        x=poses[:, 0, 3]/2000,
        y=poses[:, 1, 3]/2000,
        z=poses[:, 2, 3]/2000,
        mode='markers',
        name='cameras',
        marker=dict(size=2)
    )
)

# fig.add_trace(
#     go.Scatter3d(
#         x=end[::500, 0],
#         y=end[::500, 1],
#         z=end[::500, 2],
#         mode='markers',
#         name='ends',
#         marker=dict(size=2)
#     )
# )

n_vals = 90000
# which_cam = 1

xlines = []
ylines = []
zlines = []

sc = torch.FloatTensor(np.array([[1/2000, 0, 0],
                 [0, 1/2000, 0],
                 [0, 0, 1]]))

for which_cam in [0]:
    # rays_o, rays_d = dataset.all_rays[which_cam * n_vals:n_vals*(which_cam+1), :3], dataset.all_rays[which_cam*n_vals:n_vals*(which_cam+1), 3:6]
    # near, far = dataset.all_rays[which_cam * n_vals:n_vals*(which_cam+1), 6:7]*0.1, dataset.all_rays[which_cam*n_vals:n_vals*(which_cam+1), 7:8]*0.1


    # Fetch for the validation dataset
    val_item = dataset_val[which_cam]
    rays_o, rays_d = val_item['rays'][:, :3], val_item['rays'][:, 3:6]
    near, far = val_item['rays'][:, 6:7], val_item['rays'][:, 7:8]

    # start = (rays_o + near * rays_d).numpy()
    # end = (rays_o + far * rays_d).numpy()
    start = (rays_o @ sc + 1.0 * rays_d).numpy()
    end = (rays_o @ sc + 2.0 * rays_d).numpy()

    # xlines = []
    # ylines = []
    # zlines = []
    for i in [0, 799, -800, -1]:
        xlines += [start[i, 0], end[i, 0], None]
        ylines += [start[i, 1], end[i, 1], None]
        zlines += [start[i, 2], end[i, 2], None]
        
    for p in [[0, 799], [799, -1], [-800, -1], [-800, 0]]:
        xlines += [start[p[0], 0], start[p[1], 0], None]
        ylines += [start[p[0], 1], start[p[1], 1], None]
        zlines += [start[p[0], 2], start[p[1], 2], None]
        xlines += [end[p[0], 0], end[p[1], 0], None]
        ylines += [end[p[0], 1], end[p[1], 1], None]
        zlines += [end[p[0], 2], end[p[1], 2], None]

    # for ix in range(0, 36000, 10):
    #     xlines += [start[i, 0], end[i, 0], None]
    #     ylines += [start[i, 1], end[i, 1], None]
    #     zlines += [start[i, 2], end[i, 2], None]

fig.add_trace(
    go.Scatter3d(
        x=xlines,
        y=ylines,
        z=zlines,
        mode='lines',
        name='frustum',
        marker=dict(size=1)
    )
)

fig.update_layout(
    # scene = dict(
    #     xaxis = dict(nticks=4, range=[300,500],),
    #                  yaxis = dict(nticks=4, range=[1100,1300],),
    #                  zaxis = dict(nticks=4, range=[-1,300],),),
    width=700,
    scene_aspectmode='data',
    margin=dict(r=20, l=10, b=10, t=10))

fig.show()

In [121]:
rays_o.mean(0)

tensor([ 6.9347e+02,  3.2900e+03, -1.0000e+00])

In [22]:
dataset.all_rays.shape[0]/(800*450)

13.0

In [77]:
print(start[0], end[0])
print(start[100], end[100])

[4.1237518e+02 1.1807368e+03 8.7553257e-01] [ 418.38623   1185.4009      -1.5970472]
[4.1221692e+02 1.1809008e+03 8.4288698e-01] [ 417.5948   1186.2207     -1.760275]


In [40]:
from datasets.ray_utils import *

In [41]:
drns = get_ray_directions(450, 800, Ks[0])

In [23]:
print(sio.imread("../sus_data/even_smaller_scene/images/CAM_FRONT_RIGHT_007.jpg").shape)
# print(os.listdir("../sus_data/even_smaller_scene/images")[:10])

(225, 400, 3)


In [24]:
op = dataset_val[0]

In [28]:
val_item = dataset_val[0]
rays_o, rays_d = val_item['rays'][:, :3], val_item['rays'][:, 3:6]
near, far = val_item['rays'][:, 6:7], val_item['rays'][:, 7:8]


In [29]:
rays_o.shape, rays_d.shape, near.shape, far.shape

(torch.Size([90000]),
 torch.Size([90000, 3]),
 torch.Size([90000, 1]),
 torch.Size([90000, 1]))

In [ ]:

start = (rays_o + near * rays_d).numpy()
end = (rays_o + far * rays_d).numpy()